In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Data Preparation and Exploration

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
device

In [ ]:
def to_categorical(y, num_classes=10):
    """ 1-hot encodes a tensor """
    return np.eye(num_classes, dtype='uint8')[y]

In [ ]:
class MNISTDataset(Dataset):
    def __init__(self, root):
        super(MNISTDataset, self).__init__()
        self.df = pd.read_csv("/kaggle/input/digit-recognizer/" + root)
        self.data = self.df.to_numpy()
        self.x , self.y = self.data[:, 1:] / 255., self.data[:, 0]
        self.x = torch.from_numpy(self.x.reshape((-1, 1, 28, 28))).float()
        self.y = torch.from_numpy(to_categorical(self.y)).float()

    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx,:]
    
    def __len__(self):
        return len(self.data)

In [ ]:
VAL_SPLIT = 0.1
train_dataset = MNISTDataset("train.csv")
train_dataset_len = len(train_dataset)
train_size = int((1 - VAL_SPLIT)*train_dataset_len)
val_size = int(0.1*train_dataset_len)

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

print("Train dataset size: %d" % (len(train_dataset)))
print("Validation dataset size: %d" % (len(val_dataset)))

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

def show_image(im, label=None):
    plt.imshow(im.reshape(28, 28), cmap='gray', vmin=0, vmax=1)
    if label is not None:
        plt.title(label)
    plt.show()

In [ ]:
# for x, y in train_data_loader:
#     show_image(x[0], np.argmax(y.tolist()))
#     break

## CNN Model

In [ ]:
import albumentations as A

transformA = A.Compose([
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5)
#     A.RandomBrightnessContrast(p=0.5)
])

s = None
for x, y in train_data_loader:
    s = transformA(image=np.array(x[0].tolist()))['image']
    show_image(x[0], np.argmax(y.tolist()))
    break


In [ ]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        batch_size = x.shape[0]
        return x.view(batch_size, -1)

class Print(nn.Module):
    def forward(self, x):
        print(x.size())
        return x

class ConvNeuralNet(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.classifier = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=3, padding=2),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(0.2),
            
            nn.Conv2d(32, 128, kernel_size=3, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding=2),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(0.2),
            
            nn.Conv2d(128, 256, kernel_size=3),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(kernel_size=(2, 2)),
            nn.Dropout(0.2),
            
            nn.Flatten(),
            #Print(),
            nn.Linear(4096, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.3),

            nn.Linear(128, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
            nn.Dropout(0.5),
            
            nn.Linear(32, num_classes),
            nn.Softmax()
        ) 
    
    def forward(self, x):
        batch_size = x.shape[0]
        out = self.classifier(x)
        return out


In [ ]:
model = ConvNeuralNet(10).cuda()

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)


In [ ]:
num_epochs = 50
total_step = len(train_data_loader)
loss_list = []
acc_list = []

val_total_step = len(val_data_loader)
val_loss_list = []
val_acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        # Run the forward pass
        labels = torch.max(labels, 1)[1].cuda()
        outputs = model(images.cuda()).cuda()
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 1000 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct * 100./ total)))
    for i, (images, labels) in enumerate(val_data_loader):
        # Run the forward pass
        labels = torch.max(labels, 1)[1].cuda()
        outputs = model(images.cuda()).cuda()
        loss = criterion(outputs, labels)
        val_loss_list.append(loss.item())

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        val_acc_list.append(correct * 100./ total)

    print('Val Loss: {:.4f}, Val Accuracy: {:.2f}% \n\n\n'
          .format(np.mean(val_loss_list), np.mean(val_acc_list)))